Import of Packages

In [1]:
import scipy.stats as stats
import heapq
from collections import defaultdict
#from Code.import_data import import_data
from datetime import datetime, timedelta
import pandas as pd

Import of Data

In [2]:

def import_data():
    # @diana and denis, change to your directory
    gtfs_dir = '/Users/paulinaheine/Codes/BackupTravelPlanning/GTFS_OP_2024_obb-2/'

    # Load data
    agency_df = pd.read_csv(gtfs_dir + 'agency.txt')
    stops_df = pd.read_csv(gtfs_dir + 'stops.txt')
    routes_df = pd.read_csv(gtfs_dir + 'routes.txt')
    trips_df = pd.read_csv(gtfs_dir + 'trips.txt')
    stop_times_df = pd.read_csv(gtfs_dir + 'stop_times.txt')
    calendar_df = pd.read_csv(gtfs_dir + 'calendar.txt')
    calendar_dates_df = pd.read_csv(gtfs_dir + 'calendar_dates.txt')
    shapes_df = pd.read_csv(gtfs_dir + 'shapes.txt')
    
    # Return all DataFrames
    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df, calendar_dates_df, shapes_df

# Import and display data


Helping Functions

In [3]:
def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 60 + minutes + seconds / 60

def minutes_to_time(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

def get_weekday(date):
    weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return weekdays[date.weekday()]

def is_service_available(service_id, start_time_obj, calendar, calendar_dates):
    #date_str = date.strftime("%Y%m%d")
    #date = date.date()
    # Convert date to string in YYYYMMDD format
    date_str = start_time_obj.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][start_time_obj.weekday()]

    # Step 1: Check for exceptions in calendar_dates
    if service_id in calendar_dates["service_id"].values:
        exceptions = calendar_dates[calendar_dates["service_id"] == service_id]
        for _, exception in exceptions.iterrows():
            if exception["date"] == int(date_str):
                if exception["exception_type"] == 2:  # Service is added as an exception
                    return True
                elif exception["exception_type"] == 1:  # Service is removed as an exception
                    return False

    # Step 2: Check for regular service in calendar
    if service_id in calendar["service_id"].values:
        service = calendar[calendar["service_id"] == service_id]
        # Check if date is within start_date and end_date
        if int(service["start_date"].iloc[0]) <= int(date_str) <= int(service["end_date"].iloc[0]):
            # Check if the service operates on this weekday
            is_available = (service[weekday].iloc[0] == 1)
            if is_available == 1:
                return True
    return False

def prepare_calendar_dates(calendar_dates):
    grouped = calendar_dates.groupby("service_id")
    calendar_dates_dict = {}
    for service_id, group in grouped:
        exceptions = group.to_dict(orient="records")
        calendar_dates_dict[service_id] = exceptions
    return calendar_dates_dict

In [4]:
agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()

Create Graph

In [5]:
def create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj):
    '''
    funktioniert endlich, if not wurde entfernt weil es mich genervt hat 3-4 min 
    '''

    graph = defaultdict(list)
    stop_id_to_name = stops.set_index("stop_id")["stop_name"].to_dict()
    # Filter for active trips today using is_service
    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    trip_id_to_route = trips.set_index("trip_id")["route_id"].to_dict()

    calendar_dates_2 = prepare_calendar_dates(calendar_dates)
    stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])

    #####
    #Filter
    #####

    stop_times_copy = stop_times.copy()
    # Filter out stops outside the time window
    stop_times_copy["arrival_minutes"] = stop_times_copy["arrival_time"].apply(time_to_minutes)
    stop_times_copy["departure_minutes"] = stop_times_copy["departure_time"].apply(time_to_minutes)

    start_minutes = time_to_minutes(start_time_obj.strftime("%H:%M:%S"))
    end_minutes = time_to_minutes(end_time_obj.strftime("%H:%M:%S"))

    stop_times = stop_times_copy[
        (stop_times_copy["arrival_minutes"] >= start_minutes) &
        (stop_times_copy["departure_minutes"] <= end_minutes)
        ]

    print(f"Rows after time window filter: {len(stop_times)}")


    #trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)

    grouped = stop_times.groupby("trip_id")
    for trip_id, group in grouped:
        service_id = trip_id_to_service.get(trip_id)
        
        if is_service_available(service_id, start_time_obj, calendar, calendar_dates) == True:
    
            stops = group["stop_id"].tolist()
            departures = group["departure_time"].apply(time_to_minutes).tolist()
            arrivals = group["arrival_time"].apply(time_to_minutes).tolist()
    
            for start, end, dep, arr in zip(stops[:-1], stops[1:], departures[:-1], arrivals[1:]):
                travel_time = arr - dep
                if travel_time > 0:
                    graph[stop_id_to_name[start]].append(
                        (stop_id_to_name[end], dep, arr, trip_id_to_route.get(trip_id))
                    )

    return graph


Reliability function

In [6]:
def compute_transfer_probability_with_departure_delay(transfer_time):


    return stats.gamma.cdf(transfer_time, a=2, scale=3)


# A* 

In [7]:
import heapq
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius der Erde in Kilometern
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Entfernung in Kilometern

def heuristic(current_stop, end_name, stops_df):
    current_stop_row = stops_df[stops_df['stop_name'] == current_stop].iloc[0]
    end_stop_row = stops_df[stops_df['stop_name'] == end_name].iloc[0]

    current_lat = current_stop_row['stop_lat']
    current_lon = current_stop_row['stop_lon']
    end_lat = end_stop_row['stop_lat']
    end_lon = end_stop_row['stop_lon']

    distance_km = haversine(current_lat, current_lon, end_lat, end_lon)

    average_speed = 60  # km/h
    estimated_time_minutes = (distance_km / average_speed) * 60
    return estimated_time_minutes

#heuristic("Wien Handelskai", end_stop_name,  stops)

def a_star(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=2, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern

    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]

        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or new_current_time < best_result[0]:  # Wenn das Ziel früher erreicht wird
                best_result = (current_time, path, reliability)


        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))

    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden
    
    
def a_star_on_speed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=2, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern

    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]
 
        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))
                
        if current_stop == end_name:
            print("found")
            best_result = (current_time, path, reliability)
            
            return best_result



In [8]:
heuristic("Wien Handelskai", "Wien Rennweg",  stops)

5.300387474413974

In [9]:
'''
current_time_fast, best_result_fast , reliability_fast = a_star_on_speed_fixed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)
'''

'\ncurrent_time_fast, best_result_fast , reliability_fast = a_star_on_speed_fixed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)\n'

In [10]:
#current_time_fast, best_result_fast , reliability_fast

In [44]:


def a_star_with_reliability_fixed(graph, start_name, end_name, start_time_minutes, 
                                  exclude_routes=set(), MIN_TRANSFER_TIME=10, stops_df=None):
    pq = [(start_time_minutes, start_name, [], 1.0, None, 0)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie, Heuristik)
    visited = set()
    best_result = None  # Variable, um das beste Ergebnis zu speichern

    while pq:
        current_time, current_stop, path, reliability, last_route, _ = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]

        # Initialisierung für die Überprüfung der Verbesserungen
        no_improvement_count = 0  # Zähler für Iterationen ohne Verbesserung
        max_no_improvement = 5  # Maximale Anzahl an erlaubten Iterationen ohne Verbesserung
        
        # Überprüfe, ob das Ziel erreicht wurde
        if current_stop == end_name:
            print("possibility")
            # Hier speichern wir das erste Ergebnis, wenn das Ziel erreicht wurde
            if best_result is None or reliability > best_result[2]:
                print("possibility is better then before", best_result)
                # Wenn das Ergebnis besser ist, wird es aktualisiert
                best_result = (current_time, path, reliability) # Zähler zurücksetzen
            else:
                # Wenn keine Verbesserung, erhöhe den Zähler
                no_improvement_count += 1
        
            # Überprüfung, ob maximale Anzahl an Iterationen ohne Verbesserung erreicht wurde
            if no_improvement_count >= max_no_improvement:
                print("max ieration for here")
                return best_result  # Rückgabe des aktuellen besten Ergebnisses


        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    transfer_time = departure_time - current_time
                    if transfer_time <= MIN_TRANSFER_TIME or transfer_time > 60:
                        continue  # Zu wenig Zeit für Umstieg

                if not is_transfer:
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = compute_transfer_probability_with_departure_delay(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                # Heuristik für A* (vereinfachte Schätzung der verbleibenden Zeit)
                h = heuristic(neighbor, end_name, stops)

                # Füge den nächsten Knoten zur Priority Queue hinzu
                heapq.heappush(pq, (
                    new_current_time,  # Gesamtbewertung: aktuelle Zeit + Heuristik
                    neighbor,
                    path + [(route_id, departure_time, arrival_time)],
                    new_reliability,
                    route_id,
                    h
                ))

    # Gibt das beste Ergebnis zurück, wenn es existiert, andernfalls eine Fehlermeldung
    if best_result:
        return best_result
    else:
        return float("inf"), [], 0.0  # Keine Route gefunden


In [12]:
''''
current_time_rel, best_result_rel , reliability_rel = a_star_with_reliability_fixed(graph, start_stop_name, end_stop_name, start_time_minutes, exclude_routes=set(), MIN_TRANSFER_TIME=7)
'''

"'\ncurrent_time_rel, best_result_rel , reliability_rel = a_star_with_reliability_fixed(graph, start_stop_name, end_stop_name, start_time_minutes, exclude_routes=set(), MIN_TRANSFER_TIME=7)\n"

In [13]:
#current_time_rel, best_result_rel , reliability_rel

In [92]:


def a_star_backups(graph, path_fixed):

    backup_routes = []
    
    last_route = None
    backup_reliability = 1.0
    
    for i in range(1, len(path_fixed) - 1, 2):  # Alle Umstiegspunkte durchgehen
        transfer_stop, transfer_time = path_fixed[i - 1]
        # Identifiziere alle Umstiegspunkte in der Route
        # for i in range(1, len(path_fixed) - 1, 2):
        
    
        #adjusted_transfer_time = MIN_TRANSFER_TIME + 15
        #current_stop, current_time = path_fixed[i - 1]
        route_id, departure_time, arrival_time = path_fixed[i]
        next_stop, _ = path_fixed[i + 1]
        transfer_time_inbetween = departure_time - transfer_time
        print(f"lets go from{transfer_stop}")
        
        
        if route_id != last_route:
                missed_trans_rel = 1 - (compute_transfer_probability_with_departure_delay(transfer_time_inbetween))
                backup_time, backup_path, backup_reliability = a_star_with_reliability_fixed(
                    graph, transfer_stop, path_fixed[-1][0], start_time_minutes = departure_time + 3 , stops_df=None, MIN_TRANSFER_TIME= 8,exclude_routes=set()
                )
                backup_routes.append([backup_time, backup_path, missed_trans_rel*backup_reliability])
        last_route = route_id
        
    return backup_routes

In [89]:
best_result_fast[0]

('Wien Hauptbahnhof', 870)

878

Main

In [60]:

# Hauptprogramm
if __name__ == "__main__":

    agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()
    
    '''
    start_stop_name = "Schattendorf Kirchengasse"
    end_stop_name = "Flughafen Wien Bahnhof"
    start_datetime = "2024-12-12 14:30:00"
    '''
    
    start_stop_name = "Wien Hauptbahnhof"
    end_stop_name = "Wien Hütteldorf"
    start_datetime = "2024-12-12 14:30:00"
    

    time_budget = "3:10"
    time_budget_hours, time_budget_minutes = map(int, time_budget.split(":"))
    time_budget_minutes = time_budget_hours * 60 + time_budget_minutes / 60

    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    # In ein reines date-Objekt umwandeln
    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    end_time_obj = start_time_obj + timedelta(minutes=time_budget_minutes)
    date_obj = start_time_obj.date()

    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute


'''
    # Ergebnis anzeigen
    if current_time_fast < float("inf"):
        arrival_time_fixed = minutes_to_time(current_time_fast)
        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")

        last_route = None
        grouped_routes = []

        for i in range(1, len(best_result_fast) - 1, 2):
            current_stop, current_time = best_result_fast[i - 1]
            route_id, departure_time, arrival_time = best_result_fast[i]
            next_stop, _ = best_result_fast[i + 1]

            if route_id == last_route:
                grouped_routes[-1]["stops"].append((next_stop, arrival_time))
            else:
                grouped_routes.append({
                    "route_id": route_id,
                    "start_stop": current_stop,
                    "departure_time": departure_time,
                    "stops": [(next_stop, arrival_time)]
                })
            last_route = route_id

        for segment in grouped_routes:
            start = segment["start_stop"]
            dep_time = minutes_to_time(segment["departure_time"])
            route = segment["route_id"]
            stops = " → ".join([f"{stop} (Ankunft: {minutes_to_time(arr)})" for stop, arr in segment["stops"]])
            print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

        print(f"\n🎯 Endstation: {end_stop_name} (Ankunft: {arrival_time_fixed})")
        print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability_fast:.2f}\n")

    
    
        
        print("🔄 Backup-Routen:")
        if backup_routes:
            print("\n🔄 Backup-Routen:")
            for stop, path, reliability in backup_routes:
                print(f"  🔁 Backup von {stop}:")
                last_route = None
                first_segment = True

                for segment in path:
                    start, dep_time, end, arr_time, route = segment

                    if last_route == route:
                        print(f" → {end} (Ankunft: {arr_time})", end="")
                    else:
                        if not first_segment:
                            print()
                        print(f"    🚆 {start} (Abfahrt: {dep_time}) → {end} (Ankunft: {arr_time}) mit Linie {route}",
                              end="")
                        first_segment = False

                    last_route = route

                print(f"\n    🔹 Zuverlässigkeit: {reliability:.2f}\n")
        else:
            print("  ❌ Keine Backup-Routen verfügbar.")

        print()
    else:
        print(f"\n⚠️ Keine zuverlässige Route von {start_stop_name} nach {end_stop_name} gefunden.\n")

        '''

'\n    # Ergebnis anzeigen\n    if current_time_fast < float("inf"):\n        arrival_time_fixed = minutes_to_time(current_time_fast)\n        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")\n\n        last_route = None\n        grouped_routes = []\n\n        for i in range(1, len(best_result_fast) - 1, 2):\n            current_stop, current_time = best_result_fast[i - 1]\n            route_id, departure_time, arrival_time = best_result_fast[i]\n            next_stop, _ = best_result_fast[i + 1]\n\n            if route_id == last_route:\n                grouped_routes[-1]["stops"].append((next_stop, arrival_time))\n            else:\n                grouped_routes.append({\n                    "route_id": route_id,\n                    "start_stop": current_stop,\n                    "departure_time": departure_time,\n                    "stops": [(next_stop, arrival_time)]\n                })\n            last_route = route_id\n\n        for s

In [43]:
    #slow and correct
print("creating graph")
graph = create_graph_with_schedule(stop_times, stops, trips, calendar, calendar_dates, start_time_obj, end_time_obj)
#if start_stop_name not in graph or end_stop_name not in graph:
#    print("🚨 Ungültige Start- oder Zielhaltestelle!")
        #sys.exit()

creating graph
Rows after time window filter: 32584


/var/folders/qw/jbl_sx2j4r114n9k1myblk6c0000gn/T/ipykernel_1405/2638892618.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times["service_id"] = stop_times["trip_id"].map(trip_id_to_service)


In [51]:


    # Haupt-Dijkstra-Lauf
    #print("A star")
    #current_time_fast, best_result_fast , reliability_fast = a_star(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)

current_time_fast, best_result_fast , reliability_fast = a_star_on_speed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)




found


In [62]:
best_result_fast[1 + 1]


('Wien Matzleinsdorfer Platz', 874.0)

In [95]:
reliability_fast

np.float64(0.08875793396167544)

In [53]:
best_result_fast

[('Wien Hauptbahnhof', 870),
 ('1-S3-W-j24-1', 872.0, 874.0),
 ('Wien Matzleinsdorfer Platz', 874.0),
 ('1-S1-W-j24-1', 879.0, 881.0),
 ('Wien Hauptbahnhof', 881.0),
 ('2-R63-N-j24-1', 888.0, 891.0),
 ('Wien Grillgasse', 891.0),
 ('2-R63-N-j24-1', 891.0, 903.0),
 ('Gramatneusiedl Bahnhof', 903.0),
 ('2-R63-N-j24-1', 904.0, 916.0),
 ('Bruck/Leitha Bahnhof', 916.0),
 ('2-R63-N-j24-1', 917.0, 921.0),
 ('Parndorf Ort Bahnhof', 921.0),
 ('2-R63-N-j24-1', 922.0, 928.0),
 ('Neusiedl am See Bahnhof', 928.0),
 ('2-R64-N-j24-1', 931.0, 934.0),
 ('Jois Bahnhof', 934.0),
 ('2-R64-N-j24-1', 934.0, 936.0),
 ('Winden am See Bahnhof', 936.0),
 ('2-R64-N-j24-1', 936.0, 938.0),
 ('Breitenbrunn Bahnhof', 938.0),
 ('2-R64-N-j24-1', 938.0, 942.0),
 ('Purbach am Neusiedler See Bf.', 942.0),
 ('2-R64-N-j24-1', 946.0, 949.0),
 ('Donnerskirchen Bahnhof', 949.0),
 ('2-R64-N-j24-1', 950.0, 953.0),
 ('Schützen am Gebirge Haltestelle', 953.0),
 ('2-R64-N-j24-1', 953.0, 955.0),
 ('Schützen am Gebirge Bahnhof', 955.

In [93]:
# Backup-Routen berechnen
print("backups")
backup_routes = a_star_backups(graph,best_result_fast)

backups
lets go fromWien Hauptbahnhof
possibility
possibility is better then before None
lets go fromWien Matzleinsdorfer Platz
lets go fromWien Hauptbahnhof
lets go fromWien Grillgasse
lets go fromGramatneusiedl Bahnhof
lets go fromBruck/Leitha Bahnhof
lets go fromParndorf Ort Bahnhof
lets go fromNeusiedl am See Bahnhof
possibility
possibility is better then before None
possibility
lets go fromJois Bahnhof
lets go fromWinden am See Bahnhof
lets go fromBreitenbrunn Bahnhof
lets go fromPurbach am Neusiedler See Bf.
lets go fromDonnerskirchen Bahnhof
lets go fromSchützen am Gebirge Haltestelle
lets go fromSchützen am Gebirge Bahnhof
lets go fromEisenstadt Schule


In [98]:
backup_routes[0][2]

np.float64(0.8555236996055999)

In [108]:
def compute_itinerary_reliability(reliability_fast, backup_routes):
    """
    Berechnet die Gesamtreliability eines Itineraries.

    Parameters:
        primary_reliability: Zuverlässigkeit der primären Route (rel_i).
        backup_routes: Liste der Backup-Routen mit deren Zuverlässigkeit.

    Returns:
        Die Gesamtreliability des Itineraries.
    """
    total_reliability = reliability_fast
    for backup in backup_routes:
        total_reliability += backup[2]
    return total_reliability


In [109]:
def compute_total_reliability(reliability_fast, backup_routes):
    """
    Berechnet die Gesamtzuverlässigkeit der gesamten Route gemäß Wahrscheinlichkeitslogik.

    Parameters:
        primary_reliability: Zuverlässigkeit der primären Route (rel_i).
        backup_routes: Liste der Backup-Routen mit deren Zuverlässigkeit.

    Returns:
        Die gesamte Zuverlässigkeit der Route (rel_{\bar{i}}).
    """
    total_reliability = reliability_fast
    remaining_probability = 1 - reliability_fast  # Wahrscheinlichkeit, dass die Primärroute fehlschlägt

    for backup in backup_routes:
        backup_reliability = backup[2]
        total_reliability += backup_reliability * remaining_probability
        remaining_probability *= (1 - backup_reliability)

    return total_reliability
#zwischen 0 und 1


In [101]:
reliability_fast

np.float64(0.08875793396167544)

In [110]:
total_reliability =compute_total_reliability(reliability_fast, backup_routes)

In [103]:
total_reliability = compute_itinerary_reliability(reliability_fast, backup_routes)

In [111]:
total_reliability

np.float64(0.9652118951946763)

In [65]:
#current_time_fast, best_result_fast , reliability_fast = a_star_on_speed(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)

In [ ]:
#current_time_slow, best_result_slow , reliability_slow = a_star(graph, start_stop_name, end_stop_name, start_time_minutes,  exclude_routes=set(), MIN_TRANSFER_TIME=3)

In [ ]:
#best_result_fast

In [ ]:
#best_result_slow 

In [94]:
backup_routes

[[987.0,
  [('Wien Hauptbahnhof', 875.0),
   ('2-R63-N-j24-1', 888.0, 891.0),
   ('Wien Grillgasse', 891.0),
   ('2-R63-N-j24-1', 891.0, 903.0),
   ('Gramatneusiedl Bahnhof', 903.0),
   ('2-R63-N-j24-1', 904.0, 916.0),
   ('Bruck/Leitha Bahnhof', 916.0),
   ('2-R63-N-j24-1', 917.0, 921.0),
   ('Parndorf Ort Bahnhof', 921.0),
   ('2-R63-N-j24-1', 922.0, 928.0),
   ('Neusiedl am See Bahnhof', 928.0),
   ('2-R64-N-j24-1', 961.0, 964.0),
   ('Jois Bahnhof', 964.0),
   ('2-R64-N-j24-1', 964.0, 966.0),
   ('Winden am See Bahnhof', 966.0),
   ('2-R64-N-j24-1', 966.0, 968.0),
   ('Breitenbrunn Bahnhof', 968.0),
   ('2-R64-N-j24-1', 968.0, 972.0),
   ('Purbach am Neusiedler See Bf.', 972.0),
   ('2-R64-N-j24-1', 972.0, 975.0),
   ('Donnerskirchen Bahnhof', 975.0),
   ('2-R64-N-j24-1', 976.0, 979.0),
   ('Schützen am Gebirge Haltestelle', 979.0),
   ('2-R64-N-j24-1', 979.0, 980.0),
   ('Schützen am Gebirge Bahnhof', 980.0),
   ('2-R64-N-j24-1', 981.0, 985.0),
   ('Eisenstadt Schule', 985.0),
   